In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import os
import inception
import prettytensor as pt

In [2]:
tf.__version__

'1.1.0'

In [3]:
pt.__version__

'0.7.4'

In [4]:
import knifey

In [5]:
from knifey import num_classes

In [6]:
knifey.data_dir = "C:/Users/Aman Deep Singh/Documents/Python/Data/Knifey/data/knifey-spoony/"

In [7]:
data_dir = knifey.data_dir

In [8]:
knifey.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [9]:
dataset = knifey.load()

Creating dataset from the files in: C:/Users/Aman Deep Singh/Documents/Python/Data/Knifey/data/knifey-spoony/
- Data saved to cache-file: C:/Users/Aman Deep Singh/Documents/Python/Data/Knifey/data/knifey-spoony/knifey-spoony.pkl


In [10]:
# The code to load own image files
# from dataset import load_cached
# dataset = load_cached(cache_path='my_dataset_cache.pkl', in_dir='my_images/')
# num_classes = dataset.num_classes

In [11]:
class_names = dataset.class_names
class_names

['forky', 'knifey', 'spoony']

In [12]:
image_paths_train, cls_train, labels_train = dataset.get_training_set()

In [13]:
image_paths_train[0]

'C:\\Users\\Aman Deep Singh\\Documents\\Python\\Data\\Knifey\\data\\knifey-spoony\\forky\\forky-01-0001.jpg'

In [14]:
image_paths_test, cls_test, labels_test = dataset.get_test_set()

In [16]:
image_paths_test[0]

'C:\\Users\\Aman Deep Singh\\Documents\\Python\\Data\\Knifey\\data\\knifey-spoony\\forky\\test/forky-test-01-0001.jpg'

In [17]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(image_paths_train)))
print("- Test-set:\t\t{}".format(len(image_paths_test)))

Size of:
- Training-set:		4170
- Test-set:		530


In [18]:
def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true)

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # There may be less than 9 images, ensure it doesn't crash.
        if i < len(images):
            # Plot image.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Name of the true class.
            cls_true_name = class_names[cls_true[i]]

            # Show true and predicted classes.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Name of the predicted class.
                cls_pred_name = class_names[cls_pred[i]]

                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()